## Machine Learning
# Tarea 1 : Problema de Clasificación y de Regresión Lineal
---
__PROFESOR : Rodrigo Hernández__

__ALUMNOS : Roy Barrera - José Pérez__

---
# EXPLORACION DE DATOS 

Como se menciona en el informe adjunto, para el desarrollo de esta tarea hemos decidido trabajar con los dataset asociados a la __"VIII Encuesta de presupuestos familiares"__, que pueden ser descargados desde el siguiente enlace: https://datosabiertos.ine.cl/datasets/187191/viii-epf/

En ese mismo enlace se encuentra también la documentación de las características y tipos de variables que componen los dataset. 

En específico, usaremos:

* __BASE_PERSONAS_VIII_EPF.csv__ : con 247 variables y 48.308 observaciones.

* __BASE_GASTOS_VIII_EPF.csv__ : con 14 variables y 1.064.239 observaciones.


Dada la cantidad de variables que componen los dataset, hemos realizamos un estudio previo de la documentación de la data, para entenderla mejor y preseleccionar aquellas características candidatas a ser usadas en este trabajo académico.

Luego de varios intentos y deliberaciones escogimos analizar la data a nivel de FAMILIAS y no de individuos, ya que tanto en el problema de clasificadores como en el modelo de regresión lineal, intentamos responder preguntas relacionadas con el GASTO FAMILIAR.


### PERSONAS 

De esta forma, del dataset __BASE_PERSONAS_VIII_EPF.csv__ hemos preseleccionado las columnas que nos han parecido relevantes de las personas pero a nivel de "FAMILIA", en lugar de individuos:

| Num | Nombre | Descripcion | Comentario |
| --- | --- | --- |---|
| 1 | ZONA | Identificador de la zona geográfica | 1. Gran Santiago, 2. Resto de Capitales Regionales. |
| 2 | FOLIO_V | Identificador de vivienda | Número identificador de la vivienda encuestada |
| 3 | FOLIO | Identificador hogar | Número identificador del hogar (nucelo familiar) encuestado |
| 9 | NPERSONAS | Número de personas en el hogar | total de personas en el hogar |
| 85 | GASTOT_HD | Gasto total por hogar imputado por hot-deck (sin arriendo imputado) | suma de gastos realizados por el hogar, sin considerar el arriendo imputado|
| 95 | ING_TOTAL_HOG_HD  | Ingreso total del hogar imputado por hot-deck (sin arriendo imputado). | la sumatoria de todas las fuentes habituales de ingreso del hogar|


### GASTOS
Del mismo modo, del dataset __BASE_GASTOS_VIII_EPF.csv__ hemos preseleccionado:

| Num | Nombre | Descripcion | Comentario |
| --- | --- | --- | --- |
| 1 | ZONA | Zona Encuestada | 1 : Santiago, 2 : Otra Capital Regional |
| 2 | FOLIO_V | Folio Vivienda | Id de la vivienda encuestada |
| 3 | FOLIO | Folio | Id de la Familia encuestada |
| 4 | CCIF | Clasificacion Consumo Individual por Finalidad | Sistema internacional de clasificación |
| 5 | D | Division | 1: Alimentos y Bebidas no Alcoholicas, 2 Bebidas Alcoholicas, Tabaco, etc |
| 6 | G | Grupo | numérico,  Depende de la División |
| 7 | C | Clase | numérico, Depende de la División y del Grupo |
| 10 | GASTO | Gasto | numérico, Monto en Pesos, con decimales |
| 11 |GLOSA | Glosa | Descripción del Item CCIF |





__Nota importante__:

Para tener un mejor entendimiento del código CCIF (Clasificacion Consumo Individual por Finalidad), la base __CCIF_VIII_EPF.csv__, contiene la descripción completa de todos los códigos de gasto, abiertos por División (D), Grupo (G), Clase (C), Sub_Clase (SC) y Producto (P);  Ejemplo :

| D | G | C | SC | P | CCIF | GLOSA |
|---|---|---|---|---|---|---|
|  1 | 0 | 0 | 0 | 0 | 01.0.0.00.00 | ALIMENTOS Y BEBIDAS NO ALCOHÓLICAS |
|  2 | 0 | 0 | 0 | 0 | 02.0.0.00.00 | BEBIDAS ALCOHÓLICAS, TABACO Y ESTUPEFACIENTES |
|  3 | 0 | 0 | 0 | 0 | 03.0.0.00.00 | PRENDAS DE VESTIR Y CALZADO |
|  4 | 0 | 0 | 0 | 0 | 04.0.0.00.00 | ALOJAMIENTO, AGUA, ELECTRICIDAD, GAS Y OTROS COMBUSTIBLES |
|  5 | 0 | 0 | 0 | 0 | 05.0.0.00.00 | MUEBLES, ARTÍCULOS PARA EL HOGAR Y PARA LA CONSERVACIÓN ORDINARIA DEL HOGAR |
|  6 | 0 | 0 | 0 | 0 | 06.0.0.00.00 | SALUD |
|  7 | 0 | 0 | 0 | 0 | 07.0.0.00.00 | TRANSPORTE |
|  8 | 0 | 0 | 0 | 0 | 08.0.0.00.00 | COMUNICACIONES |
|  9 | 0 | 0 | 0 | 0 | 09.0.0.00.00 | RECREACIÓN Y CULTURA |
| 10 | 0 | 0 | 0 | 0 | 10.0.0.00.00 | EDUCACIÓN |
| 11 | 0 | 0 | 0 | 0 | 11.0.0.00.00 | RESTAURANTES Y HOTELES |
| 12 | 0 | 0 | 0 | 0 | 12.0.0.00.00 | BIENES Y SERVICIOS DIVERSOS |


Al ser el alcance de este proyecto, meramente académico, nos enfocamos solo en algunos de estos itemes de gasto

---
# EXPLORACION

La exploración de datos la hacemos de forma dirigida, según los problemas de CLASIFICADORES y REGRESION que planteamos inicialmente y que intentaremos responder posteriormente 

In [1]:
#Importacion de Librerias

#Manejo de path y carpetas
import os 

#Manejo de Data Frame
import pandas as pd

---
## 1 Dataset FAMILIA (a partir de la base de personas)

- Se lee las columnas pre-seleccionadas de la base personas (a nivel de familia), 
- Se reemplazan valores na por la media (asumiendo una distribución normal)
- Se fuerzan a enteros los valores medidos en pesos (gasto, ingreso)
- Se define una categoría ( Bajo, Medio Alto ), para la familia, según su nivel de ingreso
- Se elimina registros duplicados (la base personas contiene informacion repetida por individuos)

In [2]:
# Se lee las columnas pre-seleccionadas de la base personas (a nivel de familia), 
dataper  = 'BASE_PERSONAS_VIII_EPF.csv'
famcols  = [ 'ZONA','FOLIO','NPERSONAS','ING_TOTAL_HOG_HD','GASTOT_HD']

df_familia=pd.read_csv(dataper, sep=';', usecols=famcols, decimal=',', keep_default_na=False, na_values=[' ',-77,-88,-99] )
df_familia=df_familia.rename(columns = {'NPERSONAS':'Personas','GASTOT_HD':'Gastos','ING_TOTAL_HOG_HD':'Ingresos'})


#imputamos los datos conitnuos vacíos a la media
df_familia=df_familia.fillna(df_familia.mean())

# Se fuerzan a enteros los valores medidos en pesos (gasto, ingreso)
df_familia['Ingresos'] = df_familia['Ingresos'].round(0).astype('int64')
df_familia['Gastos'] = df_familia['Gastos'].round(0).astype('int64')

#Se define categoría ( Bajo, Medio Alto ), según ingreso
df_familia['Categoria'] = pd.cut(df_familia['Ingresos'], [0,900000, 3000000, 70000000], labels=["1.Bajo","2.Medio","3.Alto"], right=True)

#Se elimina registros duplicados
df_familia = df_familia.groupby('FOLIO', as_index=False).first()


In [3]:
df_familia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15239 entries, 0 to 15238
Data columns (total 6 columns):
FOLIO        15239 non-null object
ZONA         15239 non-null int64
Personas     15239 non-null int64
Gastos       15239 non-null int64
Ingresos     15239 non-null int64
Categoria    15169 non-null object
dtypes: int64(4), object(2)
memory usage: 833.4+ KB


In [4]:
df_familia.describe()

,ZONA,Personas,Gastos,Ingresos
count,15239.000000,15239.000000,1.523900e+04,1.523900e+04
mean,1.477984,3.170024,1.060657e+06,1.343173e+06
std,0.499531,1.623760,1.042752e+06,1.701219e+06
min,1.000000,1.000000,0.000000e+00,0.000000e+00
25%,1.000000,2.000000,4.265485e+05,4.620000e+05
50%,1.000000,3.000000,7.546520e+05,8.457000e+05
75%,2.000000,4.000000,1.318780e+06,1.572204e+06
max,2.000000,17.000000,1.897800e+07,6.208083e+07


---

## 2. Dataset GASTOS (en algunos ítems seleccionados)

- Se lee las columnas pre-seleccionadas de la base GASTOS (a nivel de familia)

In [5]:
# carga y exploracion de base Gastos (que tiene menos columnas)
datagas  = 'BASE_GASTOS_VIII_EPF.csv'
gascols  = ['ZONA','FOLIO_V','FOLIO','CCIF','D','G','C','GASTO','GLOSA']

df_gasto = pd.read_csv(datagas, sep=';', usecols=gascols, keep_default_na=False, na_values=[' ', -77], decimal=',')


In [6]:
df_gasto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1064239 entries, 0 to 1064238
Data columns (total 9 columns):
ZONA       1064204 non-null float64
FOLIO_V    1064204 non-null float64
FOLIO      1064204 non-null object
CCIF       1064237 non-null object
D          1064237 non-null float64
G          1064237 non-null float64
C          1064237 non-null float64
GASTO      1064239 non-null float64
GLOSA      1064237 non-null object
dtypes: float64(6), object(3)
memory usage: 73.1+ MB


In [7]:
df_gasto.describe()

,ZONA,FOLIO_V,D,G,C,GASTO
count,1.064204e+06,1.064204e+06,1.064237e+06,1.064237e+06,1.064237e+06,1.064239e+06
mean,1.489282e+00,7.491890e+03,4.463020e+00,2.177901e+00,2.851050e+00,1.822722e+04
std,4.998853e-01,4.329773e+03,3.933499e+00,1.734783e+00,2.350149e+00,6.358747e+04
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,1.000000e+00,3.766000e+03,1.000000e+00,1.000000e+00,1.000000e+00,2.131250e+03
50%,1.000000e+00,7.473000e+03,3.000000e+00,1.000000e+00,2.000000e+00,5.000000e+03
75%,2.000000e+00,1.126200e+04,7.000000e+00,3.000000e+00,4.000000e+00,1.343333e+04
max,2.000000e+00,1.499900e+04,1.200000e+01,8.000000e+00,9.000000e+00,1.712304e+07


---
- Se seleccionan solo algunos items de gastos (Alimentacion y Comunicaciones) para los problemas posteriores 

In [8]:
#Algunas Divisiones del Gasto, se convierten en Columnas para faciliar el trabajo posterior
    
df_aux00 = df_gasto.groupby(['FOLIO','D'], as_index=False)[['GASTO']].sum()

# 01 : ALIMENTOS Y BEBIDAS NO ALCOHÓLICAS
df_aux01=df_aux00[df_aux00['D']==1].drop(columns=['D'])
df_aux01.rename(columns={'GASTO': 'G1_Alimentacion'}, inplace=True)

# 02 : BEBIDAS ALCOHÓLICAS, TABACO Y ESTUPEFACIENTES
df_aux02=df_aux00[df_aux00['D']==2].drop(columns=['D'])
df_aux02.rename(columns={'GASTO': 'G2_Bebidas'}, inplace=True)

# 08 : COMUNICACIONES
df_aux08=df_aux00[df_aux00['D']==8].drop(columns=['D'])
df_aux08.rename(columns={'GASTO': 'G8_Comunicaciones'}, inplace=True)

# 09 : RECREACIÓN Y CULTURA
df_aux09=df_aux00[df_aux00['D']==9].drop(columns=['D'])
df_aux09.rename(columns={'GASTO': 'G9_Recreacion'}, inplace=True)

# 10 : EDUCACIÓN
df_aux10=df_aux00[df_aux00['D']==10].drop(columns=['D'])
df_aux10.rename(columns={'GASTO': 'G10_Educacion'}, inplace=True)

# 11 : RESTAURANTES Y HOTELES
df_aux11=df_aux00[df_aux00['D']==11].drop(columns=['D'])
df_aux11.rename(columns={'GASTO': 'G11_Restaurantes'}, inplace=True)



- Se realiza un merge para dar forma a un nuevo Dataframe del Gasto por Familia en los items pre seleccionados (Alimentación y Comunicaciones)

In [9]:
# merge 01 y 02
df_m01=pd.merge(df_aux01, df_aux02,  on='FOLIO') 

# merge 08 y 09
df_m02=pd.merge(df_aux08, df_aux09,  on='FOLIO') 

# merge 10 y 11
df_m03=pd.merge(df_aux10, df_aux11,  on='FOLIO') 

# merge (01,02) y (08, 09)
df_m04=pd.merge(df_m01,   df_m02,    on='FOLIO') 

# merge ( (01,02) , (08,09)  y (10, 11)
df_m05=pd.merge(df_m04,   df_m03,    on='FOLIO') 

df_m05 

,FOLIO,G1_Alimentacion,G2_Bebidas,G8_Comunicaciones,G9_Recreacion,G10_Educacion,G11_Restaurantes
0,00003-1,281100.250000,42993.125000,75693.396226,64276.382051,3.994667e+04,38146.916244
1,00005-1,255973.527878,137113.429554,110929.322637,203061.496236,4.708044e+04,148733.787228
2,00009-1,202343.931967,307.170378,145270.167519,50029.641672,9.870833e+04,24680.730814
3,00010-1,113543.777348,663.321978,40113.968095,1037.017201,9.166667e+04,11831.470055
4,00012-1,15000.000000,14100.000000,75000.000000,150000.000000,2.785000e+05,46260.869565
5,00019-1,467673.750000,20324.375000,90000.000000,369996.812500,5.109167e+05,204212.500000
6,00021-1,356125.170050,152.002639,125026.116203,418421.896134,2.171667e+05,230408.018736
7,00029-1,171942.227133,128298.666504,266450.549451,434327.298385,2.540000e+05,420299.254747
8,00031-1,184402.468414,27877.266357,75055.555556,119626.548559,2.500000e+04,64004.667343
9,00036-1,471096.042965,103929.885301,133214.125226,108317.113875,2.691667e+05,184004.261026


__NOTA IMPORTANTE__

El Merge contiene los valores del consumo pero solo para aquellas familias que GASTAN en los 4 ámbitos seleccionados.
Estas son las que nos interesa tomar como base para el análisis

---
- En el dataframe Merge se rellenan valores na que existieran utilizando el valor promedio

In [10]:
df_m05=df_m05.fillna(df_m05.mean())
df_m05

,FOLIO,G1_Alimentacion,G2_Bebidas,G8_Comunicaciones,G9_Recreacion,G10_Educacion,G11_Restaurantes
0,00003-1,281100.250000,42993.125000,75693.396226,64276.382051,3.994667e+04,38146.916244
1,00005-1,255973.527878,137113.429554,110929.322637,203061.496236,4.708044e+04,148733.787228
2,00009-1,202343.931967,307.170378,145270.167519,50029.641672,9.870833e+04,24680.730814
3,00010-1,113543.777348,663.321978,40113.968095,1037.017201,9.166667e+04,11831.470055
4,00012-1,15000.000000,14100.000000,75000.000000,150000.000000,2.785000e+05,46260.869565
5,00019-1,467673.750000,20324.375000,90000.000000,369996.812500,5.109167e+05,204212.500000
6,00021-1,356125.170050,152.002639,125026.116203,418421.896134,2.171667e+05,230408.018736
7,00029-1,171942.227133,128298.666504,266450.549451,434327.298385,2.540000e+05,420299.254747
8,00031-1,184402.468414,27877.266357,75055.555556,119626.548559,2.500000e+04,64004.667343
9,00036-1,471096.042965,103929.885301,133214.125226,108317.113875,2.691667e+05,184004.261026


---

## 3. Dataset GASTOS x FAMILIA 

- Primero : combinar los 2 Dataset anteriores, para obtener una base


In [11]:
df_t01=pd.merge(df_familia, df_m05, on='FOLIO')
df_t01

,FOLIO,ZONA,Personas,Gastos,Ingresos,Categoria,G1_Alimentacion,G2_Bebidas,G8_Comunicaciones,G9_Recreacion,G10_Educacion,G11_Restaurantes
0,00003-1,1,5,1387893,500213,1.Bajo,281100.250000,42993.125000,75693.396226,64276.382051,3.994667e+04,38146.916244
1,00005-1,2,4,1880938,2536449,2.Medio,255973.527878,137113.429554,110929.322637,203061.496236,4.708044e+04,148733.787228
2,00009-1,1,5,1399475,890000,1.Bajo,202343.931967,307.170378,145270.167519,50029.641672,9.870833e+04,24680.730814
3,00010-1,2,2,340514,87000,1.Bajo,113543.777348,663.321978,40113.968095,1037.017201,9.166667e+04,11831.470055
4,00012-1,2,1,1129143,792000,1.Bajo,15000.000000,14100.000000,75000.000000,150000.000000,2.785000e+05,46260.869565
5,00019-1,1,4,2414712,2440000,2.Medio,467673.750000,20324.375000,90000.000000,369996.812500,5.109167e+05,204212.500000
6,00021-1,1,3,2268250,3240619,3.Alto,356125.170050,152.002639,125026.116203,418421.896134,2.171667e+05,230408.018736
7,00029-1,2,4,3292790,5549383,3.Alto,171942.227133,128298.666504,266450.549451,434327.298385,2.540000e+05,420299.254747
8,00031-1,1,3,1090459,1414906,2.Medio,184402.468414,27877.266357,75055.555556,119626.548559,2.500000e+04,64004.667343
9,00036-1,2,4,2397138,1830000,2.Medio,471096.042965,103929.885301,133214.125226,108317.113875,2.691667e+05,184004.261026



- Segundo : Se filtran del dataframe temporal, solamente aquellas columnas que interesa relacionar

In [12]:
df_t02=df_t01.drop(['FOLIO','ZONA'], axis=1)
df_t02=df_t02.fillna(df_t02.mean())
df_t02=df_t02.dropna()
df_t02

,Personas,Gastos,Ingresos,Categoria,G1_Alimentacion,G2_Bebidas,G8_Comunicaciones,G9_Recreacion,G10_Educacion,G11_Restaurantes
0,5,1387893,500213,1.Bajo,281100.250000,42993.125000,75693.396226,64276.382051,3.994667e+04,38146.916244
1,4,1880938,2536449,2.Medio,255973.527878,137113.429554,110929.322637,203061.496236,4.708044e+04,148733.787228
2,5,1399475,890000,1.Bajo,202343.931967,307.170378,145270.167519,50029.641672,9.870833e+04,24680.730814
3,2,340514,87000,1.Bajo,113543.777348,663.321978,40113.968095,1037.017201,9.166667e+04,11831.470055
4,1,1129143,792000,1.Bajo,15000.000000,14100.000000,75000.000000,150000.000000,2.785000e+05,46260.869565
5,4,2414712,2440000,2.Medio,467673.750000,20324.375000,90000.000000,369996.812500,5.109167e+05,204212.500000
6,3,2268250,3240619,3.Alto,356125.170050,152.002639,125026.116203,418421.896134,2.171667e+05,230408.018736
7,4,3292790,5549383,3.Alto,171942.227133,128298.666504,266450.549451,434327.298385,2.540000e+05,420299.254747
8,3,1090459,1414906,2.Medio,184402.468414,27877.266357,75055.555556,119626.548559,2.500000e+04,64004.667343
9,4,2397138,1830000,2.Medio,471096.042965,103929.885301,133214.125226,108317.113875,2.691667e+05,184004.261026


In [13]:
df_t02.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4941 entries, 0 to 4952
Data columns (total 10 columns):
Personas             4941 non-null int64
Gastos               4941 non-null int64
Ingresos             4941 non-null int64
Categoria            4941 non-null object
G1_Alimentacion      4941 non-null float64
G2_Bebidas           4941 non-null float64
G8_Comunicaciones    4941 non-null float64
G9_Recreacion        4941 non-null float64
G10_Educacion        4941 non-null float64
G11_Restaurantes     4941 non-null float64
dtypes: float64(6), int64(3), object(1)
memory usage: 424.6+ KB


In [14]:
df_t02.describe()

,Personas,Gastos,Ingresos,G1_Alimentacion,G2_Bebidas,G8_Comunicaciones,G9_Recreacion,G10_Educacion,G11_Restaurantes
count,4941.000000,4.941000e+03,4.941000e+03,4.941000e+03,4941.000000,4941.000000,4.941000e+03,4.941000e+03,4.941000e+03
mean,3.970451,1.614312e+06,1.923799e+06,2.749368e+05,30510.643128,80189.430258,1.144853e+05,1.506017e+05,1.087676e+05
std,1.548781,1.308729e+06,2.069036e+06,1.785312e+05,42820.808602,60006.240471,1.764910e+05,2.194059e+05,1.499305e+05
min,1.000000,6.237700e+04,3.070000e+02,1.937500e+03,3.216987,3.684826,3.352893e+01,6.429251e+00,3.747312e+01
25%,3.000000,7.897710e+05,7.466670e+05,1.532214e+05,5000.000000,40113.968095,2.528150e+04,4.291667e+03,2.179672e+04
50%,4.000000,1.224790e+06,1.280000e+06,2.376860e+05,15684.408430,67458.153461,6.060895e+04,6.793332e+04,6.066429e+04
75%,5.000000,1.982535e+06,2.252900e+06,3.550237e+05,39679.998779,105031.643441,1.323217e+05,2.037500e+05,1.339603e+05
max,17.000000,1.897800e+07,3.026667e+07,2.407691e+06,517805.942127,770836.560757,2.391135e+06,2.397525e+06,2.536041e+06


- Finalmente : Salva el Dataframe como CSV, para no tener que repetir este proceso en futuras iteraciones

In [15]:
export_csv=df_t02.to_csv('Gasto_x_Familia.csv',sep=';',index=None, header= True)


## VALIDACION

Para validar el dataframe guardado, se intenta volver a leerlo y compararlo

In [16]:
df_t03 = pd.read_csv('Gasto_x_Familia.csv', sep=';', decimal='.')
df_t03.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4941 entries, 0 to 4940
Data columns (total 10 columns):
Personas             4941 non-null int64
Gastos               4941 non-null int64
Ingresos             4941 non-null int64
Categoria            4941 non-null object
G1_Alimentacion      4941 non-null float64
G2_Bebidas           4941 non-null float64
G8_Comunicaciones    4941 non-null float64
G9_Recreacion        4941 non-null float64
G10_Educacion        4941 non-null float64
G11_Restaurantes     4941 non-null float64
dtypes: float64(6), int64(3), object(1)
memory usage: 386.1+ KB


In [17]:
df_t03

,Personas,Gastos,Ingresos,Categoria,G1_Alimentacion,G2_Bebidas,G8_Comunicaciones,G9_Recreacion,G10_Educacion,G11_Restaurantes
0,5,1387893,500213,1.Bajo,281100.250000,42993.125000,75693.396226,64276.382051,3.994667e+04,38146.916244
1,4,1880938,2536449,2.Medio,255973.527878,137113.429554,110929.322637,203061.496236,4.708044e+04,148733.787228
2,5,1399475,890000,1.Bajo,202343.931967,307.170378,145270.167519,50029.641672,9.870833e+04,24680.730814
3,2,340514,87000,1.Bajo,113543.777348,663.321978,40113.968095,1037.017201,9.166667e+04,11831.470055
4,1,1129143,792000,1.Bajo,15000.000000,14100.000000,75000.000000,150000.000000,2.785000e+05,46260.869565
5,4,2414712,2440000,2.Medio,467673.750000,20324.375000,90000.000000,369996.812500,5.109167e+05,204212.500000
6,3,2268250,3240619,3.Alto,356125.170050,152.002639,125026.116203,418421.896134,2.171667e+05,230408.018736
7,4,3292790,5549383,3.Alto,171942.227133,128298.666504,266450.549451,434327.298385,2.540000e+05,420299.254747
8,3,1090459,1414906,2.Medio,184402.468414,27877.266357,75055.555556,119626.548559,2.500000e+04,64004.667343
9,4,2397138,1830000,2.Medio,471096.042965,103929.885301,133214.125226,108317.113875,2.691667e+05,184004.261026


---
# CONCLUSIONES
## de la exploración de datos

Al completar la tarea de exploración de la Data se logra constatar que, como se nos había anticipado, gran parte del esfuerzo se va en entender la data, familiarizarse con ella, filtrar, limpiar y prepararla.     

Este proceso se hace relativamente más sencillo cuando está direccionado por las preguntas que sabemos que intentaremos responder posteriormente en la parte analítica del trabajo : CLASIFICADORES y MODELO DE REGRESION LINEAL.

Habiendo concluido este arduo esfuerzo, se procede a los Notebook 2 y 3 de Análisis.

---
